<a href="https://colab.research.google.com/github/edilop/Proyecto_final_Skills_Tech/blob/main/Proyecto_Final_Forecasting_de_ventas_Walmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Skills Tech
#Proyecto Integrador - Generación 6

###Objetivo
#### En este proyecto, el alumno aplicará las habilidades obtenidas en el curso para generar un modelo que pueda hacer predicciones de ventas.

###Escenario
####Eres un científico de datos dentro de la consultoría “El oráculo”, tienes la asignación de trabajar en una empresa de retail para predecir sus ventas semanales a diferentes niveles muestrales, a nivel tienda.

##Sección 1. Obtención de datos

###1.1.Importamos las librerías

In [ ]:
#Versión de python 
!python --version

In [ ]:
#Manipulación de datos
import numpy as np
import pandas as pd
import datetime #Manipulación de tiempo
from time import process_time_ns #Medición de tiempo

#Librerías estadísticas
from statsmodels.tsa.stattools import pacf, acf #Autocorrelaciones
from scipy import stats #Prueba de Kruskal-Wallis

#Visualización de datos
import plotly.express as px #Librería de gráficos
import plotly.graph_objects as go #Librería plotly para objetos gráficos
from plotly.subplots import make_subplots #Subgráficos dentro de un aréa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf #Gráficos acf, pacf

#Seteo de opciones
pd.options.plotting.backend = "plotly" #Cambia motor de gráficas de pandas a plotly
pd.options.display.float_format = '{:,.2f}'.format #Configuramos los valores float a 2 decimales

#Machine Learning
#Instalamos librerías
!pip install pmdarima #Autoarima
!pip install neuralprophet #Neural prophet

#Importamos librerías de ML
import pmdarima #Auto arima
from sklearn.metrics import mean_squared_error #RMSE
from prophet import Prophet #Facebook prophet
from neuralprophet import NeuralProphet #Neural Prophet

###1.2.Carga del set de datos

In [ ]:
#Cargamos los datos
url = 'https://raw.githubusercontent.com/edilop/Proyecto_final_Skills_Tech/36a03076872a497869ab40e93f934a596640a665/Walmart_Store_sales.csv'
data = pd.read_csv(url, encoding='utf-8')
#Creamos el dataframe de trabajo
walmart_df = pd.DataFrame(data)

##Sección 2. Análisis Exploratorio

###2.1.Descripción de los Datos

In [ ]:
#Muestra del dataframe.
data.sample(n=10)

In [ ]:
#Dimensiones del dataframe
walmart_df.shape

In [ ]:
#Columnas del dataframe
walmart_df.columns

In [ ]:
#Metadata
walmart_df.info()

In [ ]:
#Resumen
walmart_df.describe()

In [ ]:
#Conteo de datos nulos.
walmart_df.isnull().sum()

###2.2.Análisis de relación entre variables

In [ ]:
#Dataframe de variables agrupadas por ventas
correlacion_df = walmart_df.groupby('Date').agg({'Weekly_Sales':'sum',
                                            'Holiday_Flag':'mean',
                                            'Temperature':'mean',
                                            'Fuel_Price':'mean',
                                            'CPI':'mean',
                                            'Unemployment':'mean'})\
                                            .reset_index()
#Calculamos la correlación
cor = correlacion_df.corr()

#Graficamos la correlación con un heatmap
cor_heatm = go.Figure()
cor_heatm.add_trace(go.Heatmap(z=cor,
           x=cor.columns,
           y=cor.columns,
           zmin=-1,
           zmax=1,
           colorscale=px.colors.diverging.RdBu,
           showscale=True,
           text=cor,
           texttemplate='%{text:.2f}',
           )
)
cor_heatm.update_layout(template='ggplot2', yaxis_autorange='reversed', title='Heatmap de correlación entre variables')
cor_heatm.show()

In [ ]:
#Historico de ventas de todas las sucursales                               
#Arreglo con los días festivos
fechas = ['2010-02-12', '2010-09-10', '2010-11-26', '2010-12-31',
          '2011-02-11', '2011-09-09', '2011-11-25', '2011-12-30',
          '2012-11-23', '2012-02-10',]
#Dataframe
linear = walmart_df.loc[:,['Date', 'Store', 'Weekly_Sales']]
linear = linear.groupby(['Store', 'Date']).agg({'Weekly_Sales': 'sum'})\
                                          .reset_index()
linear['Date'] = pd.to_datetime(linear.Date)
linear = linear.sort_values(by='Date')
#Graficamos el histograma de ventas de todas las sucursales
fig = px.line(linear, x='Date', y='Weekly_Sales', color='Store', 
              title='Historico de ventas de las 45 sucursales', 
              template='seaborn',)
#Trazamos los holidays como lineas negras verticales
for i in range(len(fechas)):
  fig.add_vline(x=fechas[i], line_width=3, line_dash="dash", 
                line_color="black")
fig.show()

In [ ]:
#Consolidado de ventas y boxplot.
#Dataframe
ventas_totales = walmart_df.groupby(['Date']).agg({'Weekly_Sales': 'sum'}) \
                .reset_index()
ventas_totales['Date'] = pd.to_datetime(ventas_totales.Date)
ventas_totales = ventas_totales.sort_values(by='Date')
#Graficamos el histograma y el caja de bigotes de la evolución de la ventas.
fig_totales = make_subplots(rows=1, cols=2,
                      subplot_titles=('Histórico de Ventas', 
                                      'Boxplot de Ventas'))
#Histogram
fig_totales.add_trace(go.Scatter(x=ventas_totales.Date, 
                                 y=ventas_totales.Weekly_Sales), col=1, row=1)
#Boxplot
fig_totales.add_trace(go.Box(x=ventas_totales.Weekly_Sales, boxpoints='all', 
                             boxmean=True), col=2, row=1)

#Config
fig_totales.update_layout(showlegend=False, template='seaborn', 
                          title='Histórico y Boxplot de ventas consolidadas')
fig_totales.show()

###2.3.Preguntas de Exploración

####2.3.1.¿Qué tienda tiene el máximo de ventas?

In [ ]:
#Obtenemos las 5 sucursales con más ventas
question_231 = walmart_df.groupby(['Store']).agg(
    {'Weekly_Sales': 'sum'}).sort_values(by='Weekly_Sales', 
                                        ascending=False).reset_index().head()
question_231

In [ ]:
#Gráfico de top 5 de tiendas con las mejores ventas
question_231['Store'] = question_231['Store'].astype(str)
fig_q231 = px.funnel(question_231, x='Weekly_Sales', 
                     y='Store',
                     title='Top 5 tiendas con más ventas',
                     template='seaborn',
                     color='Weekly_Sales'
                     )
fig_q231.show()


####2.3.2.¿Qué tienda tiene la desviación estándar máxima, es decir, las ventas varían mucho. Además, averigüe el coeficiente de desviación medio?

In [ ]:
#Obtenemos las 5 sucursales con mayor desviación estándar
question_232 = walmart_df.loc[:,['Store', 'Weekly_Sales']]
question_232.columns = ['Store', 'Desv_std_max']
question_232 = question_232.groupby(['Store']).std().sort_values(
    by='Desv_std_max', ascending=False).reset_index()

#Imprimimos el resultado
print(f'La sucursal con mayor desviación estándar es: {question_232.iloc[0,0]} \
con un valor de: {question_232.Desv_std_max.max():.2f}')

In [ ]:
#Calculamos el coeficiente de desviación media
#Seleccionamos la columna a analizar
column = walmart_df[walmart_df.Store == 14].loc[:,'Weekly_Sales']

#Calculamos la media y la desviación estándar
mean = column.mean()
std = column.std()

#Calculamos el coeficiente de desviación media
coef_desv_media = (std/mean)*100

#Imprimimos el resultado
print(f'El coeficiente de desviación media es de: {coef_desv_media:.2f}%')

In [ ]:
#Graficamos el resultado
fig_q232 = px.histogram(column,
                        x='Weekly_Sales',
                        y='Weekly_Sales',
                        marginal='box',
                        template='seaborn',
                        title='Distribución de ventas Store 14'
                        )
fig_q232.show()

####2.3.3.¿Qué tienda/s tiene una buena tasa de crecimiento trimestral en el tercer trimestre de 2012?

In [ ]:
#Creamos un nuevo dt para extraer la info necesaria.
quarters_df = walmart_df.loc[:,['Store', 'Date', 'Weekly_Sales']]
#Cambiamos el tipo de columna Date a tipo fecha.
quarters_df['Date'] = pd.to_datetime(quarters_df["Date"])
#Creamos la columna 'Quarters' para identififcar los trimestres
quarters_df['Quarters'] =  quarters_df['Date'].dt.quarter
#Creamos la columna 'Years' para identificar los años.
quarters_df['Years'] = quarters_df['Date'].dt.year
#Seleccionamos solo los trimestres 2 y 3
quarters_df = quarters_df[(quarters_df['Quarters'] == 2) | 
                          (quarters_df['Quarters'] == 3)]
quarters_df.head()

In [ ]:
#Creamos el dataframe de trabajo para responder la pregunta 3.
question3_df = pd.pivot_table(quarters_df,values='Weekly_Sales', 
                              index=['Store', 'Quarters'],
                              columns='Years', aggfunc='sum',)
question3_df.head()

#####2.3.3.1. Análisis de crecimientos del 3er trimestre por cada año.

In [ ]:
#Creamos el dt.
question3_df_1 = question3_df.copy().reset_index()
#Añadimos las columnas de crecimineto por cada año de trimestres 2 y 3.
# Función para calcular crecimiento entre 2do y 3er trimestre.
def crecimiento(df, anio):
  """
  Calcula las series "Tasa de crecimiento" del dataframe.
  Calcula el crecimiento porcentual entre el 2do y 3er periodo de cada una de las sucur-
  sales, y los añade a las Series de 'Tasa de crecimiento'.

  Parametros:
  df(pd.Series): Las ventas reflejadas en la serie seleccionada del dataframe.
  anio: El año de la serie en la que ocurren las ventas de df.

  Salida:
  Devuelve una serie con los cálculos de los crecimientos, 
  """
  for i in range(len(df)):
    if df.loc[i, 'Quarters'] == 3:
      df.loc[i, f'Tasa de crecimiento {anio}'] = ((df.loc[i, anio]-df.loc[i-1, anio])/
                                                  df.loc[i-1, anio])*100
    else:
      df.loc[i, f'Tasa de crecimiento {anio}'] = np.nan
  pass

#Aplicacion la función 'crecimiento' a los años 2010, 2011, 2012
crecimiento(question3_df_1, 2010)
crecimiento(question3_df_1, 2011)
crecimiento(question3_df_1, 2012)

#Reordenamos las columnas del dataframe
question3_df_1 = question3_df_1.reindex(columns=['Store', 'Quarters', 2012, 
                                                 'Tasa de crecimiento 2012',
                                                 2011, 'Tasa de crecimiento 2011', 
                                                 2010, 'Tasa de crecimiento 2010'])

question3_df_1.head()

In [ ]:
#Grafico con plotly go
fig3_1 = make_subplots(rows=3, cols=1,
                      subplot_titles=('2012','2011','2010'))

#2012
fig3_1.add_trace(
    go.Bar(x=question3_df_1['Store'], y=question3_df_1['Tasa de crecimiento 2012']),
           row=1, col=1
)

#2011
fig3_1.add_trace(
    go.Bar(x=question3_df_1['Store'], y=question3_df_1['Tasa de crecimiento 2011']),
           row=2, col=1
)

#2010
fig3_1.add_trace(
    go.Bar(x=question3_df_1['Store'], y=question3_df_1['Tasa de crecimiento 2010']),
           row=3, col=1
)

fig3_1.update_layout(showlegend=False, template='seaborn', 
                     title_text='Tasa de crecimiento en % del 3er trimestre por año por sucursal')
fig3_1.update_xaxes(tickvals=question3_df_1['Store'], zeroline=True)
fig3_1.update_yaxes(zeroline=True, zerolinecolor='black')

fig3_1.show()

##### 2.3.3.2. Análisis de crecimiento del 3er trimestre entre cada año.

In [ ]:
#Creamos el dt.
question3_df_2 = question3_df.reset_index()
question3_df_2 = question3_df_2[question3_df_2['Quarters'] == 3]
# question3_df_2 = question3_df.copy().reset_index()
question3_df_2.info()

In [ ]:
#Añadimos las columnas de crecimiento entre años.
#Crecimiento entre año 2010 a 2011.
question3_df_2['Tasa de crecimiento 2010-2011'] = ((question3_df_2[2011] - question3_df_2[2010])/question3_df_2[2010])*100
#Crecimiento entre año 2011 a 2012.
question3_df_2['Tasa de crecimiento 2011-2012'] = ((question3_df_2[2012] - question3_df_2[2011])/question3_df_2[2011])*100
#Reordenamos las columnas del dataframe.
question3_df_2 = question3_df_2.reindex(columns=['Store', 
                                                 'Quarters', 
                                                 2012, 
                                                 'Tasa de crecimiento 2011-2012', 
                                                 2011,
                                                 'Tasa de crecimiento 2010-2011', 
                                                 2010,])
question3_df_2.head()

In [ ]:
#Gráfico con plotly go
fig3_2 = make_subplots(rows=2, cols=1,
                      subplot_titles=('2011 - 2012','2010 - 2011'))
#Tasa de crecimiento 2011-2012
fig3_2.add_trace(
    go.Bar(x=question3_df_2['Store'], y=question3_df_2['Tasa de crecimiento 2011-2012']),
    row=1, col=1
)

#Tasa de crecimiento 2010-2011
fig3_2.add_trace(
    go.Bar(x=question3_df_2['Store'], y=question3_df_2['Tasa de crecimiento 2010-2011']),
    row=2, col=1
)

fig3_2.update_layout(showlegend=False, template='seaborn', 
                     title_text='Tasa de crecimiento % del 3er trimestre entre años',
                     xaxis_tickangle=-70
                     )
fig3_2.update_xaxes(tickvals=question3_df_2['Store'])
fig3_2.update_yaxes(zeroline=True, zerolinecolor='black')

fig3_2.show()

In [ ]:
#Crecimiento por ventas.
#Preparamos el dataframe
question3_df_ventas = question3_df.copy().reset_index()
question3_df_ventas = question3_df_ventas.loc[:,['Store', 'Quarters', 2012]]

#Calculamos la columna de crecimiento
for i in range(len(question3_df_ventas)):
  if question3_df_ventas.loc[i, 'Quarters'] == 3:
    question3_df_ventas.loc[i, 'Tasa de crecimiento 2012'] = (question3_df_ventas.loc[i, 2012] - question3_df_ventas.loc[i-1, 2012])
  else:
    question3_df_ventas.loc[i, 'Tasa de crecimiento 2012'] = np.nan

question3_df_ventas.head()

In [ ]:
df_1 = question3_df_ventas[question3_df_ventas['Quarters']==2].loc[:,['Store','Quarters',2012]]
df_2 = question3_df_ventas[question3_df_ventas['Quarters']==3].loc[:,['Store','Quarters',2012]]

In [ ]:
#Graficamos el crecimiento del año 2012
fig3_3 = make_subplots(specs=[[{'secondary_y':True}]])
fig3_3.add_trace(
    go.Bar(x=df_1['Store'], y=df_1[2012], name='2do trim'),
    secondary_y=False
)
#Ventas 2do trim
fig3_3.add_trace(
    go.Bar(x=df_1['Store'], y=df_2[2012], name = '3er trim'),
    secondary_y = False
)
#Ventas 3er trim
fig3_3.add_trace(
    go.Scatter(x=df_1['Store'], 
               y=question3_df_ventas['Tasa de crecimiento 2012'].dropna(),
               name = 'Crecimiento', mode = 'lines+markers', 
               line=dict(color = '#7d1a0c')),
               secondary_y = True,
)
#Diferencia entre 2do y 3er trim
fig3_3.update_layout(showlegend=True, template='seaborn', 
                     title_text='Tasa de crecimiento del 3er trimestre 2012',
                     barmode='group')

fig3_3.update_xaxes(tickvals=question3_df_ventas['Store'])
fig3_3.update_yaxes(title_text = 'Ventas', secondary_y = False)
fig3_3.update_yaxes(title_text = 'Crecimiento', secondary_y = True,
                    zeroline = True, zerolinecolor ='#7d1a0c')

fig3_3.show()

In [ ]:
question3_df_ventas.loc[question3_df_ventas['Tasa de crecimiento 2012'] == question3_df_ventas['Tasa de crecimiento 2012'].max()]

In [ ]:
question3_df_1.loc[question3_df_1['Tasa de crecimiento 2012'] == question3_df_1['Tasa de crecimiento 2012'].max()]

In [ ]:
question3_df_2.loc[question3_df_2['Tasa de crecimiento 2011-2012'] == question3_df_2['Tasa de crecimiento 2011-2012'].max()]

####2.3.4.¿Algunas festividades tienen un impacto negativo en las ventas. Averigüe los días festivos que tienen ventas más altas que las ventas medias en temporada no festiva para todas las tiendas juntas?

In [ ]:
#Creamos el frame de trabajo
question4_df = walmart_df.loc[:,['Store', 'Date','Weekly_Sales','Holiday_Flag']]
#Cambiamos la columna 'Date' a tipo fecha.
question4_df['Date'] = pd.to_datetime(question4_df['Date'], dayfirst=True)

In [ ]:
#Identificamos los días festivos.
superbowl = ['2010-02-12', '2011-02-11','2012-02-10','2013-02-08']
dia_trabajo = ['2010-09-10','2011-09-09','2012-09-07','2013-09-06']
accion_gracias=['2010-11-26','2011-11-25','2012-11-23','2013-11-29']
navidad = ['2010-12-31','2011-12-30','2012-12-28','2013-12-27']

In [ ]:
#Añadimos los días festivos al dataframe.
##Creamos la función que agregará los días festivos al dataframe.
def isHoliday (x):
  """
  Compara la fecha ingresada 'x' con varias listas, donde se encuentran almace-
  nadas las fechas de 4 diferentes días festivos, si la fecha coincide, devolve-
  rá el día festivo al que corresponda.

  Parametros:
  x(date): Fecha a buscar en las listas.

  Salida:
  Devuelve el nombre del día festivo según sea Superbolw, Día del trabajo, Acción
  de gracias o Navidad.
  """
  if x in superbowl:
    return 'Superbowl'
  elif x in dia_trabajo:
    return 'Día de Trabajo'
  elif x in accion_gracias:
    return 'Acción de gracias'
  elif x in navidad:
    return 'Navidad'
  else:
    return 'None'
##Agregamos la columna 'Festividad' al dataframe.
question4_df['Festividad'] = question4_df['Date'].dt.strftime('%Y-%m-%d').apply(isHoliday)

In [ ]:
#Comprobamos los valores de la columna 'Festividad'.
question4_df['Festividad'].value_counts()

In [ ]:
#Creamos la columna 'Years'
question4_df['Years'] = question4_df['Date'].dt.year
question4_df['Years'].value_counts()

In [ ]:
#Agrupamos los datos para obtener los valores de las ventas agrupados por fecha.
ventas_perdate = question4_df.groupby(['Date', 'Years', 'Festividad'],
                                      as_index=False).agg({'Weekly_Sales': 'sum'})
ventas_perdate.sort_values('Date')

In [ ]:
#Filtramos los días festivos.
holiday_df = ventas_perdate[ventas_perdate['Festividad'] != 'None']
holiday_df.head()

In [ ]:
#Gráfico
figura = go.Figure()
figura.add_trace(go.Scatter(x=ventas_perdate['Date'], y=ventas_perdate['Weekly_Sales'],
                         name='Ventas'))

figura.add_trace(go.Scatter(x=holiday_df['Date'], y=holiday_df['Weekly_Sales'], text=holiday_df['Festividad'],
                            mode='markers+text', name='Holidays', textposition='top center'))
figura.update_layout(template='seaborn', title='Historico de ventas totales vs Holidays')
figura.show()

######2.3.4.1.Análisis de relación entre días festivos y evolución de ventas

In [ ]:
x = ventas_perdate['Weekly_Sales'][ventas_perdate['Festividad']!='None']
y = ventas_perdate['Weekly_Sales'][ventas_perdate['Festividad']=='None']

In [ ]:
results = stats.kruskal(x,y)
x, y = results
print(f'p value = {y:.2f} > 0.05, por lo tanto, los holidays están relacionados con las ventas')

##Sección 3. Modelado

###3.1.Autoarima

In [ ]:
#Creamos el dataframe para trabajar con ARIMA.
arima_df = ventas_perdate.loc[:,['Date', 'Weekly_Sales']]
arima_df.set_index('Date', inplace=True)
arima_df.head()

In [ ]:
#Graficamos la autocorrelación de la serie origin
acf_fig = plot_acf(arima_df, lags=100)
pacf_fig = plot_pacf(arima_df, method='ywm')

In [ ]:
#Aplicamos la prueba a de Dickey-Fuller para probar estacionalidad.
adf_test = pmdarima.arima.ADFTest(alpha=0.05)
adf_test.should_diff(arima_df['Weekly_Sales'])

In [ ]:
t1_start = process_time_ns() 
#Aplicamos el modelo autoarima
stepwise_model=pmdarima.auto_arima(arima_df,
                     start_p=1,
                     start_q=1,
                     test='adf',
                     max_p=3,
                     max_q=3,
                     m=52,
                      d=1,
                      seasonal=True,
                      start_P=0,
                      D=1,
                      trace=True,
                      error_action='warn',
                      suppress_warnings=True,
                      stepwise=True,
                      n_fits=10
                     )

t1_stop = process_time_ns()

stepwise_model.summary()

In [ ]:
#Separar dataset en train y test.
train_arima = arima_df.iloc[:-30]
test_arima = arima_df.iloc[-30:]
print(train_arima.shape, test_arima.shape)

In [ ]:
#Entrenamos el modelo
stepwise_model.fit(train_arima)

In [ ]:
#Testeamos el modelo
forecast_arima = stepwise_model.predict(n_periods=len(test_arima))
forecast_arima = pd.DataFrame(forecast_arima, index=test_arima.index, columns=['Predicted_Weekly_Sales'])

#Gráfico
fig_arima = go.Figure([go.Scatter(x=test_arima.index, y=test_arima['Weekly_Sales'], name='Test Weekly Sales'),
                    go.Scatter(x=forecast_arima.index,
                               y=forecast_arima['Predicted_Weekly_Sales'], name='Predicted Weekly Sales')])
fig_arima.update_layout(template='seaborn', title='Weekly Sales vs Predicted Weekly Sales with Auto arima')
fig_arima.show()

In [ ]:
#Calculamos el error
#Función para calcular el RMSE
def rmse(actual, prediccion):
  """
  Calcula el error cuadrático medio de un modelo de ML.

  Parametros:
  actual(np.array): Array de numpy con los valores reales
  prediccion(np.array): Array de cumpy con los valores predichos por el modelo.

  Salida:
  float: El valor RMSE calculado del modelo
  """
  RMSE = mean_squared_error(actual, prediccion, squared=False)
  return RMSE

#Función para calcular MAPE
def mape(actual, prediccion):
  """
  Calcula el error porcentual absoluto medio de un modelo de ML.

  Parametros:
  actual(np.array): Array de numpy con los valores reales
  prediccion(np.array): Array de cumpy con los valores predichos por el modelo.

  Salida:
  float: El valor MAPE
  """
  actual, prediccion = np.array(actual), np.array(prediccion)
  MAPE = np.mean(np.abs(actual-prediccion)/actual) * 100
  return MAPE

In [ ]:
x =  t1_stop-t1_start

In [ ]:
#Guardamos los resultados en un dataframe
error_df = pd.DataFrame(columns = ['Modelo', 'RMSE', 'MAPE'])
error_df.loc[0,'Modelo'] = 'Autoarima'
error_df.loc[0,'RMSE'] = rmse(test_arima.Weekly_Sales, forecast_arima.Predicted_Weekly_Sales)
error_df.loc[0,'MAPE'] = mape(test_arima.Weekly_Sales, forecast_arima.Predicted_Weekly_Sales)
error_df.loc[0, 'Speed (seg)'] = x/1000000000
error_df

###3.2.Prophet

In [ ]:
#Creamos el dataframe.
prophet_df = arima_df.copy().reset_index()
#Cambiamos el nombre de las columnas por requerimiento del modelo
prophet_df.columns=['ds','y']
#Separamos los sets para entrenamiento y prueba.
train_prophet = prophet_df.iloc[:-30]
test_prophet = prophet_df.iloc[-30:]

In [ ]:
#Iniciamos la función de toma de tiempo
t2_start = process_time_ns()
#Instanciamos el objeto prophet
m = Prophet()
#Entrenamos el modelo
m.fit(train_prophet)
#Hacemos el forecast con el modelo
future = m.make_future_dataframe(periods=30, freq='W-FRI')
forecast_prophet = m.predict(future)
forecast_prophet.head()
#Terminamos la función de toma de tiempo
t2_stop = process_time_ns()
x = t2_stop-t2_start

In [ ]:
#Hacemos un plot del modelo
m.plot(forecast_prophet);

In [ ]:
#Plot de los componentes del modelo
m.plot_components(forecast_prophet);

In [ ]:
#Graficamos una comparación de los datos del test con los datos predichos por el
#modelo
fig_prophet = go.Figure([go.Scatter(x=test_prophet['ds'], y=test_prophet['y'], name='Test sales'),
                    go.Scatter(x=forecast_prophet['ds'].iloc[-30:],
                               y=forecast_prophet['yhat'].iloc[-30:], name='Predicted sales')])
fig_prophet.update_layout(template='seaborn', title='Test Sales vs Predicted Sales with Prophet')
fig_prophet.show()

In [ ]:
#Calculamos el error
#Guardamos los resultados en un dataframe
error_df.loc[1,'Modelo'] = 'Prophet'
error_df.loc[1,'RMSE'] = rmse(test_prophet.y, forecast_prophet.yhat.iloc[-30:])
error_df.loc[1,'MAPE'] = mape(test_prophet.y, forecast_prophet.yhat.iloc[-30:])
error_df.loc[1, 'Speed (seg)'] = x/1000000000
error_df

###3.3. Neural Prophet

In [ ]:
#Creamos el dataframe.
neuralp_df = prophet_df.copy()
#Separamos los sets para entrenamiento y prueba.
train_neuralp = neuralp_df.iloc[:-30]
test_neuralp = neuralp_df.iloc[-30:]

In [ ]:
#Iniciamos la función de toma de tiempo
t3_start = process_time_ns()
#Crear el objeto con el modelo.
n = NeuralProphet(growth='linear')
#Entrenamos el modelo.
metrics = n.fit(train_neuralp, freq='W-FRI')
#Hacemos el forecast con el modelo
df_future = n.make_future_dataframe(train_neuralp, periods=29, n_historic_predictions=True)
forecast_neuralp = n.predict(df_future)
#Terminamos la función de toma de tiempo
t3_stop = process_time_ns()
x = t3_stop-t3_start

In [ ]:
forecast_neuralp.head()

In [ ]:
#Hacemos un plot del modelo
n.set_plotting_backend('plotly')
fig = n.plot(forecast_neuralp);
fig.show()

In [ ]:
#Hacemos plot de los componentes del modelo
fig_components= n.plot_components(forecast_neuralp);
fig_components.show()

In [ ]:
#Hacemos plot de los parametros del modelo
fig_parameters = n.plot_parameters(forecast_neuralp);
fig_parameters.show()

In [ ]:
#Graficamos la comparación entre la predicción y los valores reales.
fig_neuralp = go.Figure([go.Scatter(x=test_neuralp['ds'], y=test_neuralp['y'], name='Test sales'),
                    go.Scatter(x=forecast_neuralp['ds'].iloc[-30:],
                               y=forecast_neuralp['yhat1'].iloc[-30:], name='Predicted sales')])
fig_neuralp.update_layout(template='seaborn', title='Test Sales vs Predicted Sales with Neural Prophet')
fig_neuralp.show()

##Sección 4. Resultados

In [ ]:
#Calculamos el error
#Guardamos los resultados en un dataframe
error_df.loc[2,'Modelo'] = 'Neural Prophet'
error_df.loc[2,'RMSE'] = rmse(test_neuralp.y, forecast_neuralp.yhat1.iloc[-30:])
error_df.loc[2,'MAPE'] = mape(test_neuralp.y, forecast_neuralp.yhat1.iloc[-30:])
error_df.loc[2, 'Speed (seg)'] = x/1000000000
error_df